In [ ]:
import sys
sys.path.append('..')
import numpy as np
import matplotlib.pyplot as plt
from hardwares.hardware_params import hardware_params
from model_analyzer import ModelAnalyzer

In [ ]:
model_id="meta-llama/Llama-2-13b-hf"
hardware="nvidia_A6000"
analyzer=ModelAnalyzer(model_id,hardware)

In [ ]:
# styles = plt.cm.Set2(range(3))
# plt.rcParams['axes.prop_cycle'] = plt.cycler('color', styles)

fig=plt.figure(figsize=(5, 5))
bar_width = 0.7

for step in ['decode','prefill']:
    plt.subplot(2, 1, 1 if step=='decode' else 2)
    weights=[]
    kvs=[]
    tmp_acts=[]
    categories=[]
    annotation_xs=[]
    annotation_texts=[]
    xs=[]
    x_st=0
    for batchsize,seqlen in [(1,1024),(1,4096),(16,1024)]:
        FP16_sum=0
        annotation_xs.append(x_st)
        annotation_texts.append(f"{step}\nbatchsize={batchsize}\n seqlen={seqlen}")
        for w,a,kv,quantization in [(16,16,16,"FP16"),(4,16,16,"W4"),(4,16,4,"W4KV4"),(4,4,4,"W4A4")]:
            print(f"batchsize={batchsize}, seqlen={seqlen}, w={w}, a={a}, kv={kv}")
            result=analyzer.analyze(batchsize,seqlen,w,a,kv)
            weights.append(result["total_results"][step]["memory_consumption_weight"])
            kvs.append(result["total_results"][step]["memory_consumption_kv_cache"])
            tmp_acts.append(result["total_results"][step]["memory_consumption_tmp_act"])
            xs.append(x_st)
            x_st+=1
            # if quantization=="W4":
            #     categories.append(f"{quantization}\nbs={batchsize}\nlen={seqlen}")
            # else:
            categories.append(f"{quantization}")
            if quantization=="FP16":
                FP16_sum=weights[-1]+kvs[-1]+tmp_acts[-1]
            weights[-1]/=FP16_sum
            kvs[-1]/=FP16_sum
            tmp_acts[-1]/=FP16_sum
        x_st+=1
    plt.bar(xs, weights, bar_width, label='Weight')
    plt.bar(xs, kvs, bar_width, bottom=weights, label='KV cache')
    plt.bar(xs, tmp_acts, bar_width, bottom=np.array(kvs) + np.array(weights), label='Tmp Act')

    for ann_x,ann_text in zip(annotation_xs,annotation_texts):
        plt.annotate(ann_text, (ann_x+2.2, 0.75), ha='center')
    plt.ylabel('Relative Memory Consumption', fontsize=9)
    plt.xticks(xs, categories, rotation=90)
    plt.legend()
    plt.tight_layout()
plt.savefig("../output/quantization_memory_consumption.pdf",bbox_inches='tight')